In [11]:
import json
import requests as rs
import requests_cache 

requests_cache.install_cache(cache_name='discogs_cache', backend='sqlite', expire_after=86400)

# Input variables
label_id = "1265272"
min_average = 4.7

In [12]:
url = "https://api.discogs.com/labels/{}/releases?page=1&per_page=25".format(label_id)
response = rs.get(url)
response_dict = json.loads(response.text)
print(json.dumps(response_dict, indent=4))

{
    "pagination": {
        "page": 1,
        "pages": 2,
        "per_page": 25,
        "items": 37,
        "urls": {
            "last": "https://api.discogs.com/labels/1265272/releases?page=2&per_page=25",
            "next": "https://api.discogs.com/labels/1265272/releases?page=2&per_page=25"
        }
    },
    "releases": [
        {
            "status": "Accepted",
            "format": "5xFile, FLAC, EP",
            "catno": "ARTSCORE 016",
            "thumb": "",
            "resource_url": "https://api.discogs.com/releases/24998860",
            "title": "The Dark Welcome",
            "id": 24998860,
            "year": 2022,
            "artist": "Tensal"
        },
        {
            "status": "Accepted",
            "format": "5xFile, MP3, EP, 320",
            "catno": "ARTSCORE 016",
            "thumb": "",
            "resource_url": "https://api.discogs.com/releases/26600165",
            "title": "The Dark Welcome",
            "id": 26600165,
          

In [13]:
# Get community release rating
response_rating_list = []
for release in response_dict["releases"]:
    resource_url = release["resource_url"]
    rating_url = resource_url + "/rating"
    response_rating = rs.get(rating_url)
    response_rating_dict = json.loads(response_rating.text)
    response_rating_list.append(response_rating_dict) 

for item in response_rating_list:
   print(item) 

{'release_id': 24998860, 'rating': {'count': 2, 'average': 4.5}}
{'release_id': 26600165, 'rating': {'count': 2, 'average': 5.0}}
{'release_id': 10446578, 'rating': {'count': 41, 'average': 4.49}}
{'release_id': 10530622, 'rating': {'count': 193, 'average': 4.58}}
{'release_id': 11058723, 'rating': {'count': 57, 'average': 4.56}}
{'release_id': 26606444, 'rating': {'count': 3, 'average': 4.33}}
{'release_id': 11184384, 'rating': {'count': 33, 'average': 4.45}}
{'release_id': 13217748, 'rating': {'count': 26, 'average': 4.27}}
{'release_id': 15163690, 'rating': {'count': 4, 'average': 4.5}}
{'release_id': 14630885, 'rating': {'count': 1, 'average': 3.0}}
{'release_id': 13862589, 'rating': {'count': 21, 'average': 4.38}}
{'release_id': 14048028, 'rating': {'count': 60, 'average': 4.43}}
{'release_id': 14298828, 'rating': {'count': 49, 'average': 4.63}}
{'release_id': 15019933, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 14651640, 'rating': {'count': 20, 'average': 4.5}}
{'rele

In [4]:
# Remove duplicates based on the "release_id" key
unique_rating_list = []
seen_ids = set()

for item in response_rating_list:
    item_id = item["release_id"]
    if item_id not in seen_ids:
        seen_ids.add(item_id)
        unique_rating_list.append(item)

for item in unique_rating_list:
   print(item) 

{'release_id': 26146235, 'rating': {'count': 0, 'average': 0.0}}
{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 10502778, 'rating': {'count': 13, 'average': 4.15}}
{'release_id': 12402975, 'rating': {'count': 10, 'average': 4.3}}
{'release_id': 17282449, 'rating': {'count': 20, 'average': 4.75}}
{'release_id': 23819831, 'rating': {'count': 17, 'average': 4.88}}
{'release_id': 23824595, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 24753029, 'rating': {'count': 13, 'average': 4.77}}
{'release_id': 26041843, 'rating': {'count': 16, 'average': 4.69}}
{'release_id': 28720261, 'rating': {'count': 8, 'average': 4.63}}
{'release_id': 29138122, 'rating': {'count': 5, 'average': 4.6}}
{'release_id': 29313661, 'rating': {'count': 0, 'average': 0.0}}


In [5]:
# Filter releases with average >= min_average
filtered_rating_list = [item for item in unique_rating_list if item["rating"]["average"] >= min_average]

for item in filtered_rating_list:
    print(item)

{'release_id': 14704507, 'rating': {'count': 14, 'average': 4.79}}
{'release_id': 17282449, 'rating': {'count': 20, 'average': 4.75}}
{'release_id': 23819831, 'rating': {'count': 17, 'average': 4.88}}
{'release_id': 23824595, 'rating': {'count': 1, 'average': 5.0}}
{'release_id': 24753029, 'rating': {'count': 13, 'average': 4.77}}


In [6]:
# Create a dictionary mapping release_id to rating_info
rating_info_dict = {item["release_id"]: item["rating"] for item in filtered_rating_list}

# Extract information from response_dict only for matching release_ids
matching_releases = [
    {
        "release_info": release_info,
        "rating_info": rating_info_dict.get(release_info["id"])
    }
    for release_info in response_dict["releases"] if release_info["id"] in rating_info_dict
]

In [7]:
# Remove duplicates based on the "id" key
unique_matching_list = []
seen_ids = set()

for item in matching_releases:
    item_id = item["release_info"]["id"]
    if item_id not in seen_ids:
        seen_ids.add(item_id)
        unique_matching_list.append(item)

for release in unique_matching_list:
    print("Release Info:")
    print(json.dumps(release["release_info"], indent=4))
    print("Rating Info:")
    print(json.dumps(release["rating_info"], indent=4))
    print()

Release Info:
{
    "status": "Accepted",
    "format": "12\", EP",
    "catno": "LIP003",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/14704507",
    "title": "Euphoria",
    "id": 14704507,
    "year": 2020,
    "artist": "Ferdinger"
}
Rating Info:
{
    "count": 14,
    "average": 4.79
}

Release Info:
{
    "status": "Accepted",
    "format": "12\", EP",
    "catno": "LIP004",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/17282449",
    "title": "Spektrum",
    "id": 17282449,
    "year": 2021,
    "artist": "Various"
}
Rating Info:
{
    "count": 20,
    "average": 4.75
}

Release Info:
{
    "status": "Accepted",
    "format": "12\", W/Lbl, Sta",
    "catno": "LIP005",
    "thumb": "",
    "resource_url": "https://api.discogs.com/releases/23819831",
    "title": "Piano Phase",
    "id": 23819831,
    "year": 2022,
    "artist": "Ferdinger"
}
Rating Info:
{
    "count": 17,
    "average": 4.88
}

Release Info:
{
    "status": "Acc

In [8]:
# Create specific output with changed URL
output_list = []
for release in unique_matching_list:
    new_dict = {
        "catno": release["release_info"]["catno"],
        "artist": release["release_info"]["artist"],
        "title": release["release_info"]["title"],
        "year": release["release_info"]["year"],
        "format": release["release_info"]["format"],
        "url": "https://www.discogs.com/es/release/{}".format(release["release_info"]["id"]),
        "rating": release["rating_info"]
    }
    output_list.append(new_dict)

# print(output_list)
print(json.dumps(output_list, indent=4))

[
    {
        "catno": "LIP003",
        "artist": "Ferdinger",
        "title": "Euphoria",
        "year": 2020,
        "format": "12\", EP",
        "url": "https://www.discogs.com/es/release/14704507",
        "rating": {
            "count": 14,
            "average": 4.79
        }
    },
    {
        "catno": "LIP004",
        "artist": "Various",
        "title": "Spektrum",
        "year": 2021,
        "format": "12\", EP",
        "url": "https://www.discogs.com/es/release/17282449",
        "rating": {
            "count": 20,
            "average": 4.75
        }
    },
    {
        "catno": "LIP005",
        "artist": "Ferdinger",
        "title": "Piano Phase",
        "year": 2022,
        "format": "12\", W/Lbl, Sta",
        "url": "https://www.discogs.com/es/release/23819831",
        "rating": {
            "count": 17,
            "average": 4.88
        }
    },
    {
        "catno": "LIP005",
        "artist": "Ferdinger",
        "title": "Piano Phase",
  